In [1]:
# import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import math
from sklearn import svm

In [56]:
# read excel and geenrate dataframe
np.random.seed(3)
raw_dataset = pd.DataFrame(pd.read_excel(r'dataset_watermelon3.xlsx'))
raw_dataset = raw_dataset.sample(frac = 1).reset_index(drop=True) # shuffle
raw_dataset.loc[raw_dataset['好瓜']=='是','好瓜']=1
raw_dataset.loc[raw_dataset['好瓜']=='否','好瓜']=0
print(raw_dataset)

# one-hot encoding
watermelon_dataset = pd.DataFrame()
for col in raw_dataset.iloc[:,:-3].columns :
    for element in raw_dataset[col].unique():
        watermelon_dataset[col+element]=np.zeros_like(raw_dataset[col])
        watermelon_dataset.loc[raw_dataset[col]==element,col+element]=1
for col in raw_dataset.iloc[:,-3:].columns :
    watermelon_dataset[col]=raw_dataset[col]
    
print(watermelon_dataset)


    色泽  根蒂  敲声  纹理  脐部  触感     密度    含糖率  好瓜
0   青绿  蜷缩  沉闷  稍糊  稍凹  硬滑  0.719  0.103   0
1   乌黑  稍蜷  浊响  稍糊  稍凹  软粘  0.481  0.149   1
2   浅白  蜷缩  浊响  清晰  凹陷  硬滑  0.556  0.215   1
3   乌黑  稍蜷  浊响  清晰  稍凹  硬滑  0.437  0.211   1
4   乌黑  稍蜷  浊响  清晰  稍凹  软粘  0.360  0.370   0
5   乌黑  蜷缩  沉闷  清晰  凹陷  硬滑  0.774  0.376   1
6   浅白  蜷缩  浊响  模糊  平坦  软粘  0.343  0.099   0
7   乌黑  蜷缩  浊响  清晰  凹陷  硬滑  0.634  0.264   1
8   浅白  稍蜷  沉闷  稍糊  凹陷  硬滑  0.657  0.198   0
9   青绿  稍蜷  浊响  清晰  稍凹  软粘  0.403  0.237   1
10  青绿  蜷缩  浊响  清晰  凹陷  硬滑  0.697  0.460   1
11  青绿  稍蜷  浊响  稍糊  凹陷  硬滑  0.639  0.161   0
12  浅白  蜷缩  浊响  模糊  平坦  硬滑  0.593  0.042   0
13  青绿  蜷缩  沉闷  清晰  凹陷  硬滑  0.608  0.318   1
14  青绿  硬挺  清脆  清晰  平坦  软粘  0.243  0.267   0
15  乌黑  稍蜷  沉闷  稍糊  稍凹  硬滑  0.666  0.091   0
16  浅白  硬挺  清脆  模糊  平坦  硬滑  0.245  0.057   0
    色泽青绿  色泽乌黑  色泽浅白  根蒂蜷缩  根蒂稍蜷  根蒂硬挺  敲声沉闷  敲声浊响  敲声清脆  纹理稍糊  纹理清晰  纹理模糊  \
0      1     0     0     1     0     0     1     0     0     1     0     0   
1      0     1     0     0     1  

In [57]:
Iris_dataset = pd.read_table('iris.data', delimiter=',',header=None)
Iris_dataset = Iris_dataset.sample(frac = 1).reset_index(drop=True) # shuffle
# print(Iris_dataset[4].unique())
# ['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']
category_dic = {}
for i,category in enumerate(Iris_dataset[4].unique()):
    Iris_dataset.loc[Iris_dataset[4]==category,4]=i
    category_dic[i] = category
print(Iris_dataset)

       0    1    2    3  4
0    5.9  3.2  4.8  1.8  0
1    6.9  3.1  5.4  2.1  1
2    4.6  3.4  1.4  0.3  2
3    4.6  3.1  1.5  0.2  2
4    4.6  3.2  1.4  0.2  2
5    6.7  3.1  5.6  2.4  1
6    6.8  2.8  4.8  1.4  0
7    7.4  2.8  6.1  1.9  1
8    6.0  3.0  4.8  1.8  1
9    5.0  3.5  1.3  0.3  2
10   5.7  4.4  1.5  0.4  2
11   6.1  2.6  5.6  1.4  1
12   4.5  2.3  1.3  0.3  2
13   7.7  3.0  6.1  2.3  1
14   5.0  3.0  1.6  0.2  2
15   4.4  3.2  1.3  0.2  2
16   5.4  3.4  1.5  0.4  2
17   6.7  3.3  5.7  2.5  1
18   4.9  2.4  3.3  1.0  0
19   6.1  3.0  4.9  1.8  1
20   5.6  2.8  4.9  2.0  1
21   6.4  3.2  5.3  2.3  1
22   6.7  3.0  5.2  2.3  1
23   6.2  2.2  4.5  1.5  0
24   5.8  2.6  4.0  1.2  0
25   6.4  3.2  4.5  1.5  0
26   5.9  3.0  5.1  1.8  1
27   5.4  3.9  1.7  0.4  2
28   5.5  2.3  4.0  1.3  0
29   6.3  2.5  5.0  1.9  1
..   ...  ...  ...  ... ..
120  6.5  2.8  4.6  1.5  0
121  6.0  2.2  5.0  1.5  1
122  5.2  3.4  1.4  0.2  2
123  6.5  3.0  5.5  1.8  1
124  6.0  2.9  4.5  1.5  0
1

In [58]:
# define the class
# about multi-classification:  https://blog.csdn.net/u013719780/article/details/78312165
# using OVO(One-Vs-One) here
# 均使用了OvO的多分类方法
class LDA_Iris:
    
    def __init__(self,dataset):
        self.dataset = dataset
        
    
    def training(self,data_matrix_0,data_matrix_1):
        u0 = data_matrix_0.mean(0) # [1,8]
        u1 = data_matrix_1.mean(0)
        
        #def compute_sigma(self):
        tmp0 = np.zeros_like((data_matrix_0[0,:]-u0).dot((data_matrix_0[0,:]-u0).T))
        for raw in range(data_matrix_0.shape[0]):
            tmp0 += (data_matrix_0[raw,:]-u0).dot((data_matrix_0[raw,:]-u0).T)
        sigma0=1/(data_matrix_0.shape[0]-1) * tmp0
        
        tmp1 = np.zeros_like((data_matrix_1[0,:]-u1).dot((data_matrix_1[0,:]-u1).T))
        for raw in range(data_matrix_1.shape[0]):
            tmp1 += (data_matrix_1[raw,:]-u1).dot((data_matrix_1[raw,:]-u1).T)
        sigma1=1/(data_matrix_1.shape[0]-1) * tmp1
        
        S_w = sigma0 + sigma1
        
        # def compute_w(self):

        if isinstance(S_w,np.float64):
            w = (1/S_w) * (u0-u1)
        else:
            w = np.linalg.inv(S_w).dot(u0-u1)
        
        class_0_centre =  w.T.dot(u0)
        class_1_centre =  w.T.dot(u1)
        
        return w,class_0_centre,class_1_centre # [1,8]
    
    def  classification(self,sample,w,class_i_centre,class_j_centre,i,j):
        
        return i if abs(w.T.dot(sample) - class_j_centre)> \
                abs(w.T.dot(sample) - class_i_centre) else j
    
    def Cross_validation(self, k=5):
        num_in_each_group = int(len(self.dataset)/k)
        correct_validation = 0
        wrong_validation = 0
        for i in range(0,k):
            k_train_set = pd.concat([self.dataset.iloc[0:i*num_in_each_group,:],  \
                                self.dataset.iloc[(i+1)*num_in_each_group:,:]])

            k_class_0_set = (k_train_set[k_train_set[4]==0].values)[:,:-1]
            k_class_1_set = (k_train_set[k_train_set[4]==1].values)[:,:-1]
            k_class_2_set = (k_train_set[k_train_set[4]==2].values)[:,:-1]

            k_val_set = self.dataset.iloc[i*num_in_each_group:(i+1)*num_in_each_group,:]

            # using OVO 
            w_01,class_0_centre_01,class_1_centre_01 = self.training(k_class_0_set,k_class_1_set)
            w_02,class_0_centre_02,class_2_centre_02 = self.training(k_class_0_set,k_class_2_set)
            w_12,class_1_centre_12,class_2_centre_12 = self.training(k_class_1_set,k_class_2_set)
            
            
            for j in range(len(k_val_set.index)):
                vector_x = k_val_set.iloc[j,:-1].values # [1,8]
                groud_truth = k_val_set.iloc[j,-1]

                predict_value_01 = self.classification(vector_x,w_01,class_0_centre_01,class_1_centre_01,0,1)
                predict_value_02 = self.classification(vector_x,w_02,class_0_centre_02,class_2_centre_02,0,2)
                predict_value_12 = self.classification(vector_x,w_12,class_1_centre_12,class_2_centre_12,1,2)
                
                if predict_value_01 == predict_value_02:
                    predict_value = 0
                elif predict_value_01 == predict_value_12:
                    predict_value = 1
                elif predict_value_12 == predict_value_02:
                    predict_value = 2
                else:
                    predict_value = np.random.choice([0,1,2])
                
                
                if groud_truth==predict_value:
                    correct_validation+=1
                else:
                    wrong_validation+=1
        return float(correct_validation/(wrong_validation+correct_validation))
            
    

In [59]:
class LDA_watermelon:
    def __init__(self,dataset):
        self.dataset = dataset
        
    def training(self,data_matrix_0,data_matrix_1):
        u0 = data_matrix_0.mean(0) # [1,8]
        u1 = data_matrix_1.mean(0)
        
        #def compute_sigma(self):
        tmp0 = np.zeros_like((data_matrix_0[0,:]-u0).dot((data_matrix_0[0,:]-u0).T))
        for raw in range(data_matrix_0.shape[0]):
            tmp0 += (data_matrix_0[raw,:]-u0).dot((data_matrix_0[raw,:]-u0).T)
        sigma0=1/(data_matrix_0.shape[0]-1) * tmp0
        
        tmp1 = np.zeros_like((data_matrix_1[0,:]-u1).dot((data_matrix_1[0,:]-u1).T))
        for raw in range(data_matrix_1.shape[0]):
            tmp1 += (data_matrix_1[raw,:]-u1).dot((data_matrix_1[raw,:]-u1).T)
        sigma1=1/(data_matrix_1.shape[0]-1) * tmp1
        
        S_w = sigma0 + sigma1
        
        # def compute_w(self):

        if isinstance(S_w,np.float64):
            w = (1/S_w) * (u0-u1)
        else:
            w = np.linalg.inv(S_w).dot(u0-u1)
        
        class_0_centre =  w.T.dot(u0)
        class_1_centre =  w.T.dot(u1)
        
        return w,class_0_centre,class_1_centre # [1,8]
    
    def  classification(self,sample,w,class_0_centre,class_1_centre):
        
        return 1 if abs(w.T.dot(sample) - class_0_centre)> \
                abs(w.T.dot(sample) - class_1_centre) else 0
    
    def Cross_validation(self, k=5):
        num_in_each_group = int(len(self.dataset)/k)
        correct_validation = 0
        wrong_validation = 0
        for i in range(0,k):
            k_train_set = pd.concat([self.dataset.iloc[0:i*num_in_each_group,:],  \
                                self.dataset.iloc[(i+1)*num_in_each_group:,:]])

            k_class_0_set = (k_train_set[k_train_set['好瓜']==0].values)[:,:-1]
            k_class_1_set = (k_train_set[k_train_set['好瓜']==1].values)[:,:-1]

            k_val_set = self.dataset.iloc[i*num_in_each_group:(i+1)*num_in_each_group,:]


            w,class_0_centre,class_1_centre = self.training(k_class_0_set,k_class_1_set)
            
            
            for j in range(len(k_val_set.index)):
                vector_x = k_val_set.iloc[j,:-1].values # [1,8]
                groud_truth = k_val_set.iloc[j,-1]

                predict_value = self.classification(vector_x,w,class_0_centre,class_1_centre)
                if groud_truth==predict_value:
                    correct_validation+=1
                else:
                    wrong_validation+=1
        return float(correct_validation/(wrong_validation+correct_validation))
        
    

In [60]:
# define the class
class Naive_Bayes_watermelon:
    
    def __init__(self,dataset):
        self.dataset = dataset
            
        
    def MLE(self,sample_density,sample_sugar_content,class_i_dataset):
        def compute_probability(x,mu,sigma):
            exp = math.exp(-(x-mu)**2/(2*(sigma**2)))
            coe = 1/(((2*math.pi)**0.5)*sigma)
            return coe*exp
        density = class_i_dataset[:,-3] # [N,1]
        sugar_content = class_i_dataset[:,-2] # [N,1]
        
        mu_d = density.mean()
        mu_s = sugar_content.mean()
        
        sigma_d = np.std(density)
        sigma_s = np.std(sugar_content)
        
        return (compute_probability(sample_density, mu_d, sigma_d),
                        compute_probability(sample_sugar_content, mu_s, sigma_s) )
        
    def classification(self,sample_property,train_classification,class_0_set,class_1_set):

        num_of_train_set = len(train_classification)
        num_of_class_0 = len(train_classification[train_classification==0])
        class_0_probability = 1*num_of_class_0/num_of_train_set
        for i in range(len(sample_property)-2):
            class_0_probability *= len(class_0_set[class_0_set[:,i]==sample_property[i]]) \
                                    /num_of_class_0
        
        p_den_0,p_sug_0 = self.MLE(sample_property[-2],sample_property[-1],class_0_set)
        class_0_probability = class_0_probability*p_den_0*p_sug_0
        
        num_of_class_1 = len(train_classification[train_classification==1])
        class_1_probability = 1*num_of_class_1/num_of_train_set
        for i in range(len(sample_property)-2):
            class_1_probability *= len(class_1_set[class_1_set[:,i]==sample_property[i]]) \
                                    /num_of_class_1
        p_den_1,p_sug_1 = self.MLE(sample_property[-2],sample_property[-1],class_1_set)
        class_1_probability = class_1_probability*p_den_1*p_sug_1
        
        return (1 if class_0_probability<class_1_probability else 0)   

    def Cross_validation(self, k=5):
        num_in_each_group = int(len(self.dataset)/k)
        correct_classification = 0
        wrong_classification = 0

        for i in range(0,k):
            
            k_train_set = pd.concat([self.dataset.iloc[0:i*num_in_each_group,:],  \
                                self.dataset.iloc[(i+1)*num_in_each_group:,:]])
            k_train_classification =  k_train_set.iloc[:,-1].values
            k_class_0_set = k_train_set[k_train_set['好瓜']==0].values 
            k_class_1_set = k_train_set[k_train_set['好瓜']==1].values 

            k_val_set = self.dataset.iloc[i*num_in_each_group:(i+1)*num_in_each_group,:]
            k_val_property = k_val_set.iloc[:,:-1].values

            for j in range((k_val_property.shape[0])) :
                if(k_val_set.iloc[j,-1]==   \
                        self.classification(k_val_property[j,:],k_train_classification,k_class_0_set,k_class_1_set)):
                    correct_classification += 1
                else :
                    wrong_classification += 1
        # print(correct_classification,wrong_classification)
        return correct_classification/(correct_classification+wrong_classification)

    

In [61]:
# define the class
class Naive_Bayes_Iris:
    
    def __init__(self,dataset):
        self.dataset = dataset
            
        
    def MLE(self,sample_character,i,class_i_dataset):
        def compute_probability(x,mu,sigma):
            exp = math.exp(-(x-mu)**2/(2*(sigma**2)))
            coe = 1/(((2*math.pi)**0.5)*sigma)
            return coe*exp
        set_character = class_i_dataset[:,i] # [N,1]
        
        mu= set_character.mean()
        
        sigma= np.std(set_character)

        
        return compute_probability(sample_character, mu, sigma)
                        
        
    def classification(self,sample_property,train_classification,class_0_set,class_1_set,class_2_set):

        num_of_train_set = len(train_classification)
        num_of_class_0 = len(train_classification[train_classification==0])
        class_0_probability = 1*num_of_class_0/num_of_train_set
        for i in range(len(sample_property)):
            class_0_probability *= self.MLE(sample_property[i],i,class_0_set)
            # print(i)
        #-----------------------------------------------------------------------
        num_of_class_1 = len(train_classification[train_classification==1])
        class_1_probability = 1*num_of_class_1/num_of_train_set
        for i in range(len(sample_property)):
            class_1_probability *= self.MLE(sample_property[i],i,class_1_set)

        #-----------------------------------------------------------------------
        num_of_class_2 = len(train_classification[train_classification==2])
        class_2_probability = 1*num_of_class_2/num_of_train_set
        for i in range(len(sample_property)):
            class_2_probability *= self.MLE(sample_property[i],i,class_2_set)
        

        if class_2_probability == max(class_2_probability,class_1_probability,class_0_probability):
            return 2
        elif class_1_probability == max(class_2_probability,class_1_probability,class_0_probability):
            return 1
        else :
            return 0

    def Cross_validation(self, k=5):
        num_in_each_group = int(len(self.dataset)/k)
        correct_classification = 0
        wrong_classification = 0

        for i in range(0,k):
            
            k_train_set = pd.concat([self.dataset.iloc[0:i*num_in_each_group,:],  \
                                self.dataset.iloc[(i+1)*num_in_each_group:,:]])
            k_train_classification =  k_train_set.iloc[:,-1].values
            k_class_0_set = k_train_set[k_train_set[4]==0].values 
            k_class_1_set = k_train_set[k_train_set[4]==1].values 
            k_class_2_set = k_train_set[k_train_set[4]==2].values 

            k_val_set = self.dataset.iloc[i*num_in_each_group:(i+1)*num_in_each_group,:]
            k_val_property = k_val_set.iloc[:,:-1].values

            for j in range((k_val_property.shape[0])) :
                if(k_val_set.iloc[j,-1]==   \
                        self.classification(k_val_property[j,:],k_train_classification,k_class_0_set,k_class_1_set,k_class_2_set)):
                    correct_classification += 1
                else :
                    wrong_classification += 1
        # print(correct_classification,wrong_classification)
        return correct_classification/(correct_classification+wrong_classification)

    

In [62]:
class SVM_method:
    def __init__(self,dataset):
        self.dataset = dataset

        
    def train(self,train_matrix,train_classification,kernel=None):
        model = svm.SVC(kernel=kernel)
        model.fit(X=train_matrix, y=train_classification,sample_weight=None)
        return model
                
        
    def cernel_comparasion(self):
        kernel_list = ['linear','poly','rbf','sigmoid']
        for kernel_ in kernel_list:
            self.Cross_validation(kernel_)
    
    def Cross_validation(self, kernel, k=5):
        num_in_each_group = int(len(self.dataset)/k)
        correct_classification = 0
        wrong_classification = 0

        for i in range(0,k):
            
            k_train_set = pd.concat([self.dataset.iloc[0:i*num_in_each_group,:],  \
                                self.dataset.iloc[(i+1)*num_in_each_group:,:]])
            k_train_classification =  k_train_set.iloc[:,-1].values
            k_train_set = k_train_set.iloc[:,:-1].values
            
            k_val_set = self.dataset.iloc[i*num_in_each_group:(i+1)*num_in_each_group,:]
            k_val_property = k_val_set.iloc[:,:-1].values

            model = self.train(k_train_set, k_train_classification, kernel)

            for j in range((k_val_property.shape[0])) :

                if(k_val_set.iloc[j,-1]==   \
                         model.predict(k_val_property[None,j,:])):
                    correct_classification += 1
                else :
                    wrong_classification += 1
        # print(correct_classification,wrong_classification)
        accuracy = correct_classification/(correct_classification+wrong_classification)
        print(('Accuracy with Kernel:'+ kernel+':',"%.2f%%"  %(accuracy*100)))

In [63]:
class Logistic_Regression_Iris:
    def __init__(self,dataset):
        self.dataset = dataset

    def Cross_validation(self, k=5):
        num_in_each_group = int(len(self.dataset)/k)
        correct_classification = 0
        wrong_classification = 0

        for i in range(0,k):
            
            k_train_set = pd.concat([self.dataset.iloc[0:i*num_in_each_group,:],  \
                                self.dataset.iloc[(i+1)*num_in_each_group:,:]])
            k_class_0_set = (k_train_set[k_train_set[4]==0].values)[:,:-1]
            k_class_1_set = (k_train_set[k_train_set[4]==1].values)[:,:-1]
            k_class_2_set = (k_train_set[k_train_set[4]==2].values)[:,:-1]

            k_val_set = self.dataset.iloc[i*num_in_each_group:(i+1)*num_in_each_group,:]
            k_val_property = k_val_set.iloc[:,:-1].values

            k_w01, k_b01 = self.Gradient_Descent(k_class_1_set,k_class_0_set) #1->1, 0->0
            k_w02, k_b02 = self.Gradient_Descent(k_class_2_set,k_class_0_set) #2->1, 0->0
            k_w12, k_b12 = self.Gradient_Descent(k_class_1_set,k_class_2_set) #1->1, 2->0

            for j in range((k_val_property.shape[0])) :

                if(k_val_set.iloc[j,-1]==   \
                        self.LR_OvO(k_val_property[j,:], k_w01, k_b01, k_w02, k_b02, k_w12, k_b12)):
                    correct_classification += 1
                else :
                    wrong_classification += 1
        # print(correct_classification,wrong_classification)
        return correct_classification/(correct_classification+wrong_classification)
    
    def Gradient_Descent(self,class_1_set,class_0_set,learning_rate = 0.1):
        def p_y_is_1(w,x,b):
            tmp = math.exp(w.dot(x)+b)
            # print(tmp)
            return tmp/(1+tmp)
        w = np.ones_like(class_1_set[0])
        b = 1
        prev_w = w
        prev_b = b

        while True:
            prev_w = w
            prev_b = b
            sigma_1 = 0
            sigma_0 = 0
            for i in range(class_1_set.shape[0]):
                sigma_1 += class_1_set[i] * p_y_is_1(prev_w, class_1_set[i], prev_b)
            for i in range(class_0_set.shape[0]):
                sigma_0 += class_0_set[i] * p_y_is_1(prev_w, class_0_set[i], prev_b)
            w = prev_w + learning_rate * (class_1_set.sum(0) -sigma_1-sigma_0 )

            sigma_1 = 0
            sigma_0 = 0
            for i in range(class_1_set.shape[0]):
                sigma_1 += p_y_is_1(prev_w, class_1_set[i], prev_b)
            for i in range(class_0_set.shape[0]):
                sigma_0 += p_y_is_1(prev_w, class_0_set[i], prev_b)
            b = prev_b + learning_rate * (class_1_set.shape[0] -sigma_1-sigma_0 )
            if abs(sum(prev_w-w))<1e-2:
                break

        return w,b
    
    def LR_OvO(self, val_property, k_w01, k_b01, k_w02, k_b02, k_w12, k_b12):

        compare01 =  (1 if 1/(math.exp(k_w01.dot(val_property)+k_b01)+1)<0.5 else 0)
        compare02 =  (2 if 1/(math.exp(k_w02.dot(val_property)+k_b02)+1)<0.5 else 0)
        compare12 =  (1 if 1/(math.exp(k_w12.dot(val_property)+k_b12)+1)<0.5 else 2)

        if compare01==compare02:
            return 0
        elif compare01==compare12:
            return 1
        elif compare02==compare12:
            return 2
        else :
            return np.random.choice([0,1,2])


In [64]:
class Logistic_Regression_watermelon:
    def __init__(self,dataset):
        self.dataset = dataset

    def Cross_validation(self, k=5):
        num_in_each_group = int(len(self.dataset)/k)
        correct_classification = 0
        wrong_classification = 0

        for i in range(0,k):
            
            k_train_set = pd.concat([self.dataset.iloc[0:i*num_in_each_group,:],  \
                                self.dataset.iloc[(i+1)*num_in_each_group:,:]])
            k_class_0_set = (k_train_set[k_train_set['好瓜']==0].values)[:,:-1]
            k_class_1_set = (k_train_set[k_train_set['好瓜']==1].values)[:,:-1]

            k_val_set = self.dataset.iloc[i*num_in_each_group:(i+1)*num_in_each_group,:]
            k_val_property = k_val_set.iloc[:,:-1].values

            k_w01, k_b01 = self.Gradient_Descent(k_class_1_set,k_class_0_set) #1->1, 0->0

            for j in range((k_val_property.shape[0])) :

                if(k_val_set.iloc[j,-1]==   \
                        (1 if 1/(math.exp(k_w01.dot(k_val_property[j,:])+k_b01)+1)<0.5 else 0)):
                    correct_classification += 1
                else :
                    wrong_classification += 1
        # print(correct_classification,wrong_classification)
        return correct_classification/(correct_classification+wrong_classification)
    
    def Gradient_Descent(self,class_1_set,class_0_set,learning_rate = 0.1):
        def p_y_is_1(w,x,b):
            tmp = math.exp(w.dot(x)+b)
            # print(tmp)
            return tmp/(1+tmp)
        w = np.ones_like(class_1_set[0])
        b = 1
        prev_w = w
        prev_b = b

        while True:
            prev_w = w
            prev_b = b
            sigma_1 = 0
            sigma_0 = 0
            for i in range(class_1_set.shape[0]):
                sigma_1 += class_1_set[i] * p_y_is_1(prev_w, class_1_set[i], prev_b)
            for i in range(class_0_set.shape[0]):
                sigma_0 += class_0_set[i] * p_y_is_1(prev_w, class_0_set[i], prev_b)
            w = prev_w + learning_rate * (class_1_set.sum(0) -sigma_1-sigma_0 )

            sigma_1 = 0
            sigma_0 = 0
            for i in range(class_1_set.shape[0]):
                sigma_1 += p_y_is_1(prev_w, class_1_set[i], prev_b)
            for i in range(class_0_set.shape[0]):
                sigma_0 += p_y_is_1(prev_w, class_0_set[i], prev_b)
            b = prev_b + learning_rate * (class_1_set.shape[0] -sigma_1-sigma_0 )
            if abs(sum(prev_w-w))<1e-2:
                break

        return w,b
    


In [65]:
# with LDA method:
print('Using LDA method:')
LDA_watermelon_classification = LDA_watermelon(watermelon_dataset)
print('Accuracy on Watermelon validation set:',"%.2f%%"  %(LDA_watermelon_classification.Cross_validation()*100))
LDA_Iris_classification = LDA_Iris(Iris_dataset)
print('Accuracy on Iris validation set:',"%.2f%%"  %(LDA_Iris_classification.Cross_validation()*100))

# with Bayes method:
print('\nUsing Naive Bayes method:')
Naive_Bayes_watermelon_classification = Naive_Bayes_watermelon(watermelon_dataset)
print('Accuracy on Watermelon validation set:',"%.2f%%"  %(Naive_Bayes_watermelon_classification.Cross_validation()*100))
Naive_Bayes_Iris_classification = Naive_Bayes_Iris(Iris_dataset)
print('Accuracy on Iris validation set:',"%.2f%%"  %(Naive_Bayes_Iris_classification.Cross_validation()*100))

# with SVM method:
print('\nUsing SVM method:')
print('On watermelon dataset:')
SVM_watermelon_classification = SVM_method(watermelon_dataset)
SVM_watermelon_classification.cernel_comparasion( )
print('On Iris dataset:')
SVM_Iris_classification = SVM_method(Iris_dataset)
SVM_Iris_classification.cernel_comparasion()

# with Logistic Regression method:
print('\nUsing Logistic Regression method:')
Logistic_Regression_watermelon_classification = Logistic_Regression_watermelon(watermelon_dataset)
print('Accuracy on Watermelon validation set:',"%.2f%%"  %(Logistic_Regression_watermelon_classification.Cross_validation()*100))
Logistic_Regression_Iris_classification = Logistic_Regression_Iris(Iris_dataset)
print('Accuracy on Iris validation set:',"%.2f%%"  %(Logistic_Regression_Iris_classification.Cross_validation()*100))


Using LDA method:
Accuracy on Watermelon validation set: 73.33%
Accuracy on Iris validation set: 90.67%

Using Naive Bayes method:
Accuracy on Watermelon validation set: 66.67%
Accuracy on Iris validation set: 94.00%

Using SVM method:
On watermelon dataset:
('Accuracy with Kernel:linear:', '73.33%')
('Accuracy with Kernel:poly:', '60.00%')
('Accuracy with Kernel:rbf:', '66.67%')
('Accuracy with Kernel:sigmoid:', '60.00%')
On Iris dataset:
('Accuracy with Kernel:linear:', '98.67%')
('Accuracy with Kernel:poly:', '97.33%')
('Accuracy with Kernel:rbf:', '98.00%')
('Accuracy with Kernel:sigmoid:', '22.67%')

Using Logistic Regression method:
Accuracy on Watermelon validation set: 66.67%
Accuracy on Iris validation set: 97.33%
